In [19]:
import numpy as np
import pandas as pd
import random
import time
from sklearn.model_selection import RepeatedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy import linalg
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
import requests

In [20]:
def EncoderLDA(A):
    begin1 = time.time()
    U, s, VT = np.linalg.svd(A)
    end1 = time.time()
    newx=([])
    dmax=50
    tmp = np.zeros([dmax,1])
    ASEKNN=[]
#Split Data
    rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=5)
    for train_index, test_index in rkf.split(A):
         #print("TRAIN:", train_index, "TEST:", test_index)
         X_train, X_test = A[train_index], A[test_index]
         y_train, y_test = y[train_index], y[test_index]
         X1 = X_train.transpose()
         X_train, X_test = X1[train_index], X1[test_index]
         for d in range (1,dmax):
            newx=U[:,0:d]*s[0:d]**0.5
            #newx=np.reshape(newx,(n,d))
            clf2 = LinearDiscriminantAnalysis()
            newx_train, newx_test =newx[train_index], newx[test_index]
            clf2.fit(newx_train, y_train.ravel())
            tmp[d,0]=tmp[d,0]+clf2.score(newx_test, y_test.ravel())/50
         
         neigh2 = KNeighborsClassifier(weights='distance',metric='euclidean')
         neigh2.fit(newx_train, y_train.ravel())
         ASEKNN.append(neigh2.score(newx_test, y_test))
    db=tmp.argmax()
    ASELDAACC=tmp.max()
    ASELDATIME=end1-begin1
    ASEKNNACC=sum(ASEKNN)/50
#nk,w,Z
    begin2 = time.time()
    AEEKNNACC=[]
    AEELDAACC=[]
    AEEKNN2ACC=[]
    rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=5)
    for train_index, test_index in rkf.split(A):
         #print("TRAIN:", train_index, "TEST:", test_index)
         X_train, X_test = A[train_index], A[test_index]
         y_train, y_test = y[train_index], y[test_index]
         X1 = X_train.transpose()
         X_train, X_test = X1[train_index], X1[test_index]
            
         begin4 = time.time()
         nk = np.zeros((1,K))
         for i in range(0,len(y_train)-1):
                nk[0,int(y_train[i,0]-1)]=nk[0,int(y_train[i,0]-1)]+1
                w = np.zeros((int(np.size(y_train)),K))
         for i in range(0,int(np.size(y_train))):
                k=int(y_train[i])
                w[i][k-1]=1/nk[0,k-1]
         z_train=np.matmul(X_train,w)
         z_test = np.matmul(X_test,w)
         end4 = time.time()
         zTIME = end4-begin4

         begin2 = time.time()
         neigh = KNeighborsClassifier(weights='uniform',metric='euclidean')
         neigh.fit(z_train, y_train.ravel())
         AEEKNNACC.append(neigh.score(z_test, y_test))
         end2 = time.time()
         AEEKNNTIME=end2-begin2
            
         neigh3 = KNeighborsClassifier(weights='distance',metric='euclidean')
         neigh.fit(z_train, y_train.ravel())
         AEEKNN2ACC.append(neigh.score(z_test, y_test))
            
         begin3 = time.time()
         clf4 = LinearDiscriminantAnalysis()
         clf4.fit(z_train,y_train.ravel())
         AEELDAACC.append(clf4.score(z_test, y_test.ravel()))
         end3 = time.time()
         AEELDATIME=end3-begin3
#Get result
    AeK=sum(AEEKNNACC)/50
    AeL=sum(AEELDAACC)/50
    Acc=np.array([ASELDAACC,ASEKNNACC,AeK,AeL])
    Time=np.array([ASELDATIME,AEEKNNTIME+zTIME,AEELDATIME+zTIME])
    print(f"d = {db}")
    print(f"Accuracy for each method: {Acc}")
    print(f"Time for each method: {Time}")